TL:DR Les dates du jeu de données demandent du travail pour être exploitables et au final l'information n'a pas d'impact sur la fréquentation.
On a peu d'espoir de voir ces données jouer un rôle significatif dans le modèle de prédiction.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime, os
import re

In [2]:
df = pd.read_csv('244400404_effectifs-eleves-cleaned')
df.head()

,RowNo,Type de site,Date,Prévision,Réel,Nom du site,An,RéelMed,RéelNorm
0,0,M/E,2011-09-06,290.0,271.0,LEDRU ROLLIN/SARAH BERNHARDT,2011,308.0,0.879870
1,1,M/E,2011-09-05,95.0,102.0,BEAUJOIRE,2011,129.0,0.790698
2,2,M/E,2011-09-05,167.0,171.0,CONTRIE,2011,211.0,0.810427
3,3,M,2011-09-05,80.0,64.0,COUDRAY M.,2011,77.0,0.831169
4,4,M,2011-09-05,73.0,77.0,FONTENY,2011,92.0,0.836957


In [3]:
df['datetime']= df['Date'].apply( lambda string1 :datetime.datetime.strptime(string1,"%Y-%m-%d"))
df['joursem']= df['Date'].apply( lambda string1 :datetime.datetime.strptime(string1,"%Y-%m-%d").weekday())
df['isoweek']= df['Date'].apply( lambda string1 :datetime.datetime.strptime(string1,"%Y-%m-%d").isocalendar().week)

In [4]:
df.head()

,RowNo,Type de site,Date,Prévision,Réel,Nom du site,An,RéelMed,RéelNorm,datetime,joursem,isoweek
0,0,M/E,2011-09-06,290.0,271.0,LEDRU ROLLIN/SARAH BERNHARDT,2011,308.0,0.879870,2011-09-06,1,36
1,1,M/E,2011-09-05,95.0,102.0,BEAUJOIRE,2011,129.0,0.790698,2011-09-05,0,36
2,2,M/E,2011-09-05,167.0,171.0,CONTRIE,2011,211.0,0.810427,2011-09-05,0,36
3,3,M,2011-09-05,80.0,64.0,COUDRAY M.,2011,77.0,0.831169,2011-09-05,0,36
4,4,M,2011-09-05,73.0,77.0,FONTENY,2011,92.0,0.836957,2011-09-05,0,36


In [5]:
df_calend=pd.read_csv('../../data/DataGithub/fetes_juives.csv',sep=';',encoding='latin1')
# df_calend=pd.read_csv('../../data/DataGithub/fetes_juives_veille.csv',sep=';',encoding='latin1')
df_calend['datetime']= df_calend['Date'].apply(lambda string1 : datetime.datetime.strptime(string1,"%d/%m/%Y"))
df_calend.drop(columns='Date',inplace=True)
df_calend.head()

,Fete Juive,datetime
0,Pourim,2011-03-21
1,Pessah,2011-04-20
2,Chavouot,2011-06-09
3,Jeûne,2011-08-10
4,Roch,2011-09-30


On vérifie que toutes les fêtes sont bien là.

In [6]:
df_calend['Fete Juive'].value_counts()

Pourim       11
Pessah       11
Chavouot     11
Jeûne        11
Roch         11
Yom          11
Soukkot      11
Simhat       11
Hanoukkah    11
Name: Fete Juive, dtype: int64

In [7]:
df=pd.merge(df, df_calend, how='left', on='datetime')
df['Fete Juive'].fillna('-', inplace=True)
df.head()

,RowNo,Type de site,Date,Prévision,Réel,Nom du site,An,RéelMed,RéelNorm,datetime,joursem,isoweek,Fete Juive
0,0,M/E,2011-09-06,290.0,271.0,LEDRU ROLLIN/SARAH BERNHARDT,2011,308.0,0.879870,2011-09-06,1,36,-
1,1,M/E,2011-09-05,95.0,102.0,BEAUJOIRE,2011,129.0,0.790698,2011-09-05,0,36,-
2,2,M/E,2011-09-05,167.0,171.0,CONTRIE,2011,211.0,0.810427,2011-09-05,0,36,-
3,3,M,2011-09-05,80.0,64.0,COUDRAY M.,2011,77.0,0.831169,2011-09-05,0,36,-
4,4,M,2011-09-05,73.0,77.0,FONTENY,2011,92.0,0.836957,2011-09-05,0,36,-


In [8]:
df[['Fete Juive','RéelNorm']].groupby('Fete Juive').mean()

,RéelNorm
Fete Juive,
-,0.991371
Chavouot,0.983129
Hanoukkah,1.015665
Pessah,0.985087
Pourim,1.017640
Roch,1.043234
Simhat,1.037225
Soukkot,1.039746
Yom,1.023178


On n'observe pas vraiment d'effet notable d'une fête ou d'une autre.

In [9]:
df[['Fete Juive','RéelNorm']].groupby('Fete Juive').median()

,RéelNorm
Fete Juive,
-,1.000000
Chavouot,0.986347
Hanoukkah,1.004407
Pessah,0.995652
Pourim,1.000000
Roch,1.028275
Simhat,1.039139
Soukkot,1.022347
Yom,1.021309


In [10]:
df_calend1 = df[['Nom du site','Fete Juive','RéelNorm']].groupby(['Nom du site','Fete Juive']).median()
df_calend2= pd.pivot_table(df_calend1, index=['Nom du site'],columns='Fete Juive', values='RéelNorm', aggfunc=np.mean)
df_calend2.sort_values(by='Yom', ascending=True).head(30)

Fete Juive,-,Chavouot,Hanoukkah,Pessah,Pourim,Roch,Simhat,Soukkot,Yom
Nom du site,,,,,,,,,
CHATAIGNIERS,0.843478,0.878261,0.982609,NaN,0.820460,0.834479,NaN,0.816880,0.705882
GUSTAVE ROCH,1.000000,0.999493,0.956790,NaN,0.987500,1.042338,NaN,1.030286,0.833333
JEAN ZAY M,1.000000,0.953488,1.038462,NaN,1.302326,1.137179,NaN,1.166667,0.846154
CAMILLE CLAUDEL,1.000000,0.967232,0.982759,1.000000,1.039276,1.000000,1.071083,0.960784,0.872727
JEAN ZAY,1.000000,1.025351,0.957341,NaN,0.948572,0.925926,NaN,1.020768,0.875000
CHATAIGNIERS M,1.000000,0.974026,0.986667,NaN,0.974026,1.053333,NaN,1.026667,0.893333
ALAIN FOURNIER,1.000000,0.894908,1.097711,NaN,1.051603,0.994800,NaN,1.034624,0.923611
CHAUVINIERE M,1.000000,0.907692,1.019692,1.056604,1.123077,1.005892,1.088889,0.992308,0.932437
JACQUES PREVERT,1.000000,0.957189,1.034483,0.861446,1.060188,1.042017,1.106760,0.950355,0.943262


In [11]:
df_calend2.describe()

Fete Juive,-,Chavouot,Hanoukkah,Pessah,Pourim,Roch,Simhat,Soukkot,Yom
count,104.000000,104.000000,103.000000,85.000000,102.000000,102.000000,85.000000,102.000000,101.000000
mean,0.998435,0.978111,1.017573,0.987553,1.014573,1.039192,1.043830,1.033559,1.022294
std,0.016035,0.070466,0.070359,0.089196,0.084117,0.061677,0.133369,0.056849,0.099319
min,0.843478,0.569231,0.928571,0.715686,0.781609,0.834479,0.704708,0.816880,0.705882
25%,1.000000,0.950165,0.983485,0.954902,0.976323,1.008253,0.992168,1.007093,0.993326
50%,1.000000,0.986477,1.008030,0.995652,1.000000,1.031213,1.045205,1.026113,1.021043
75%,1.000000,1.013566,1.030435,1.041958,1.034814,1.059158,1.078072,1.045565,1.047347
max,1.013460,1.259494,1.523077,1.170213,1.500000,1.266429,1.984615,1.273036,1.769231


In [12]:
pd.pivot_table( df[['Fete Juive','An','RéelNorm']].groupby(['Fete Juive','An']).median(), index=['Fete Juive'],columns='An', values='RéelNorm', aggfunc=np.mean)

An,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Fete Juive,,,,,,,,,,,,
-,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.004348,1.000000,1.0
Chavouot,0.989849,NaN,1.057543,0.998901,NaN,0.939890,0.996897,NaN,NaN,NaN,0.972442,NaN
Hanoukkah,NaN,0.969112,1.052608,1.098129,1.021159,NaN,1.005629,0.935252,NaN,NaN,0.992951,NaN
Pessah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.995652,NaN
Pourim,0.988742,1.020579,NaN,0.974026,NaN,1.000000,0.971429,NaN,1.043032,NaN,NaN,NaN
Roch,1.025141,1.023888,1.010385,1.045326,0.989691,1.072677,1.059937,1.006676,1.047458,NaN,NaN,NaN
Simhat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.060435,1.012030,NaN
Soukkot,1.039572,1.004975,1.028778,1.026547,1.003247,1.076765,1.054795,1.006735,1.003030,NaN,NaN,NaN
Yom,NaN,0.995477,NaN,NaN,0.973618,1.105566,NaN,1.013698,1.013475,1.036103,1.053295,NaN


In [13]:
pd.pivot_table( df[['Fete Juive','An','RéelNorm']].groupby(['Fete Juive','An']).mean(), index=['Fete Juive'],columns='An', values='RéelNorm', aggfunc=np.mean)

An,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Fete Juive,,,,,,,,,,,,
-,1.008331,1.002772,1.006862,1.008866,0.998611,1.003982,0.994415,0.989812,0.976986,0.939041,0.985548,0.95046
Chavouot,0.975672,NaN,1.069701,1.004265,NaN,0.911504,0.982831,NaN,NaN,NaN,0.954138,NaN
Hanoukkah,NaN,0.979424,1.084513,1.124694,1.010690,NaN,0.996986,0.937832,NaN,NaN,0.968872,NaN
Pessah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.985087,NaN
Pourim,0.994251,1.049108,NaN,0.994375,NaN,1.027948,0.967172,NaN,1.073418,NaN,NaN,NaN
Roch,1.059163,1.037276,1.037738,1.065643,0.987240,1.061453,1.076550,1.001768,1.061348,NaN,NaN,NaN
Simhat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.057866,1.016584,NaN
Soukkot,1.066576,1.017197,1.066045,1.056069,0.999862,1.057728,1.085120,1.004065,1.003683,NaN,NaN,NaN
Yom,NaN,1.014679,NaN,NaN,0.938836,1.089211,NaN,1.007383,1.009714,1.021158,1.081689,NaN


A priori, il n'y a pas d'impact des fêtes juives sur la fréquentation des cantines. On peut garder l'information, on s'attend à avoir un contrôle négatif qui ne jouera pas un rôle important dans le modèle.

In [14]:
# df_calend.to_csv('calendrier_fetes_juives.csv')